In [52]:
import geopandas as gpd
import pandas as pd
import shapely.geometry as sg
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [53]:
df_geral = {}

# Lendo os arquivos de 2019 a 2022 

for i in range(2001, 2013): 
    df_geral[i] = pd.read_csv(f'C:/Users/Nathalia/Documents/UFPB/PIBIC 2024/Dados/Codigo/Dados/s{i}.txt', sep='\t')

    # Retirando as linhas que possuem coordenadas nulas
    df_geral[i] = df_geral[i].drop(df_geral[i][df_geral[i]['COORD.GEO.X'] == 0].index)
    df_geral[i] = df_geral[i].drop(df_geral[i][df_geral[i]['COORD.GEO.Y'] == 0].index)

   # Filtrando pela província de La Victoria 
    df_geral[i] = df_geral[i][df_geral[i]['DISTRITO'] == 'LA VICTORIA']


df_geral[2012].head(10)

C:\Users\Nathalia\AppData\Local\Temp\ipykernel_13172\3558385826.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geral[i] = pd.read_csv(f'C:/Users/Nathalia/Documents/UFPB/PIBIC 2024/Dados/Codigo/Dados/s{i}.txt', sep='\t')


,NRO_CLIENT,DISTRITO,TARIFA,CATEGORIA,CNS_ACT_FP,CNS_ACT_HP,SET,ALIMENTAD,SED,SEC_TIPICO,COORD.GEO.X,COORD.GEO.Y,MES
0,511,LA VICTORIA,BT5B,0,338.8,0.0,G,13,S-00433,1,279517.90,8666061.70,12
9,1230,LA VICTORIA,BT5B,0,239.9,0.0,C,19,S-00331,1,280230.44,8664222.03,12
13,1263,LA VICTORIA,BT5B,0,674.2,0.0,G,18,S-00036,1,279400.19,8665300.19,12
15,1280,LA VICTORIA,BT5B,0,431.5,0.0,G,13,S-00433,1,279484.79,8666131.13,12
19,1636,LA VICTORIA,BT5B,0,172.5,0.0,SL,3,S-00303,1,281826.22,8665574.42,12
38,3097,LA VICTORIA,BT5B,0,0.0,0.0,SL,3,C-08414,1,281571.00,8665738.00,12
40,3105,LA VICTORIA,BT5B,0,873.0,0.0,G,14,C-07743,1,279451.17,8666408.91,12
48,1413552,LA VICTORIA,BT5B,0,114.4,0.0,C,21,S-00270,1,281352.38,8665314.09,12
57,4520,LA VICTORIA,BT5B,0,122.5,0.0,SL,4,S-00744,1,281964.39,8665599.01,12
71,1418762,LA VICTORIA,BT5B,0,64.0,0.0,C,21,S-00270,1,281337.84,8665449.71,12


In [54]:
# Fazendo Pontos a partir das coordenadas, isso é necessário para o geopandas
# criar a coluna geometry do Geodataframe

points = {}

for i in range(2001, 2013):
    points[i] = df_geral[i].apply(lambda row: sg.Point(row['COORD.GEO.X'], row['COORD.GEO.Y']), axis=1)


In [55]:
geodf = {}

for i in range(2001, 2013):
    # Criando os Geodataframes a partir do dataframe e dos pontos criados
    geodf[i] = gpd.GeoDataFrame(df_geral[i], geometry=points[i])
    # Setando o sistema de coordenadas para o UTM 18S, que é o sistema de coordenadas dos dados
    geodf[i].crs = '32718'
    # Convertendo para o sistema de coordenadas geográficas WGS84
    # Isso é necessário para o plotly conseguir plotar os pontos
    geodf[i] = geodf[i].to_crs('EPSG:4326')

In [56]:
# Subtraindo 0.0022 do x e 0.0032 do y para que os pontos fiquem mais centralizados
# Obs: Isso é necessário pois o plotly não está plotando os pontos no local correto
# Obs2: Não sei porque isso está acontecendo, mas acredito que seja alguma conversão de sistema de coordenadas
# Obs3: Esses valores foram encontrados na tentativa e erro, não sei se são os valores corretos
for i in range(2001, 2013):
    geodf[i]['geometry'] = geodf[i]['geometry'].apply(lambda x: sg.Point(x.x - 0.0022, x.y - 0.0032))

In [57]:
# Os quadrados escolhidos para análise foram os 296 e 316 do outro código 
square_216 = [
    sg.Point(-77.0019956353078, -12.06870647082062), 
    sg.Point(-77.0019956353078, -12.065700086444597), 
    sg.Point(-77.00500201968381, -12.065700086444597), 
    sg.Point(-77.00500201968381, -12.06870647082062), 
    sg.Point(-77.0019956353078, -12.06870647082062)
]

square_316 = [
    sg.Point(-76.99898925093177, -12.06870647082062), 
    sg.Point(-76.99898925093177, -12.065700086444597), 
    sg.Point(-77.0019956353078, -12.065700086444597),
    sg.Point(-77.0019956353078, -12.06870647082062), 
    sg.Point(-76.99898925093177, -12.06870647082062)
    ]

In [58]:
# Irei filtrar os geodataframes de forma a restar apenas as linhas que possuem 
# pontos dentro do quadrado 216 e 316, além de criar a coluna "quadrado" que
# indica em qual quadrado o ponto está

for i in range(2001, 2013):
    geodf[i]['quadrado'] = geodf[i]['geometry'].apply(lambda x: '216' if sg.Point(x.x, x.y).within(sg.Polygon(square_216)) else '316' if sg.Point(x.x, x.y).within(sg.Polygon(square_316)) else 'outro')
    geodf[i] = geodf[i][geodf[i]['quadrado'] != 'outro']

In [80]:
# Juntarei os geodataframes de 2001 a 2012 em um único geodataframe, adicionando a coluna "datetime"
# que indica o ano e mês, representado pela coluna "MES", da linha no formato AAAA-MM

geodf_geral = gpd.GeoDataFrame()

for i in range(2001, 2013):
    geodf[i]["ANO"] = i

dfs_to_concat = [geodf[i] for i in range(2001, 2013)]
geodf_geral = gpd.GeoDataFrame(pd.concat(dfs_to_concat, ignore_index=True))

geodf_geral['datetime'] = geodf_geral.apply(lambda row: f'{row["ANO"]}-{row["MES"]}', axis=1)


In [81]:
# Excluindo as colunas que não serão utilizadas
geodf_geral = geodf_geral.drop(columns=['COORD.GEO.Y', 'COORD.GEO.X', 'geometry', 'DISTRITO']) # Deixei mês e ano que Hyago não usou
geodf_geral.head()

,NRO_CLIENT,TARIFA,CATEGORIA,CNS_ACT_FP,CNS_ACT_HP,SET,ALIMENTAD,SED,SEC_TIPICO,MES,quadrado,ANO,datetime
0,19762,BT5B,0,145.0,0.0,SL,3,S-00304,1,1,216,2001,2001-1
1,18711,BT5B,0,382.3,0.0,SL,3,S-00305,1,1,316,2001,2001-1
2,24617,BT5B,7,4933.1,0.0,SL,3,S-00305,1,1,316,2001,2001-1
3,1423425,BT5B,0,102.5,0.0,SL,3,C-08901,1,1,216,2001,2001-1
4,48851,BT5B,0,125.0,0.0,SL,4,A-03646,1,1,216,2001,2001-1


In [75]:
# Filtrarei o geodataframe de forma a restar apenas os clientes que aparecem em todos
# os meses, ou seja, que possuem 48 linhas, que é o número de meses de 2001 a 2012

#geodf_geral = geodf_geral.groupby('NRO_CLIENT').filter(lambda x: len(x) == 144)

In [82]:
geodf_geral.to_csv('geodf_geral.csv', index=False)

In [83]:
geodf_geral = pd.read_csv('geodf_geral.csv')
geodf_geral.head()

,NRO_CLIENT,TARIFA,CATEGORIA,CNS_ACT_FP,CNS_ACT_HP,SET,ALIMENTAD,SED,SEC_TIPICO,MES,quadrado,ANO,datetime
0,19762,BT5B,0,145.0,0.0,SL,3.0,S-00304,1,1,216,2001,2001-1
1,18711,BT5B,0,382.3,0.0,SL,3.0,S-00305,1,1,316,2001,2001-1
2,24617,BT5B,7,4933.1,0.0,SL,3.0,S-00305,1,1,316,2001,2001-1
3,1423425,BT5B,0,102.5,0.0,SL,3.0,C-08901,1,1,216,2001,2001-1
4,48851,BT5B,0,125.0,0.0,SL,4.0,A-03646,1,1,216,2001,2001-1


In [84]:
# Filtrando pela tarifa BT5B e pela categoria 0.0, consumo até 500 kWh

geodf_geral = geodf_geral[geodf_geral['TARIFA'] == ' BT5B']
geodf_geral = geodf_geral[geodf_geral['CATEGORIA'] == 0.0]

In [ ]:
# Hora ponto - Hora de mais energia: 17:30 + 3h 
# Hora fora ponto - Tudo que não estiver na hora ponto


# Somar os dois para ver o consumo total do dia (criar coluna com esse valor)
# 0.01 para diferenciar no gráfico que foi um dado faltante 

In [90]:
geodf_geral["NRO_CLIENT"].value_counts()

# Ver depois qual o numero máximo de contagens de um cliente que existiu

NRO_CLIENT
19762      12
1316500    12
1733371    12
1572879    12
1751491    12
           ..
1683016     6
817144      5
1978734     5
817226      2
1992716     1
Name: count, Length: 1080, dtype: int64

In [88]:
# Obtendo a Series de contagens
contagens = geodf_geral["NRO_CLIENT"].value_counts()

# Obtendo a contagem máxima
max_contagem = contagens.max()

# Obtendo a contagem mínima
min_contagem = contagens.min()

print(f'Maior contagem: {max_contagem}')
print(f'Menor contagem: {min_contagem}')

Maior contagem: 12
Menor contagem: 1


In [87]:
# Vou calcular a média de todos os clientes durante todo o período e colocar em um dataframe
# Para isso, vou agrupar por NRO_CLIENTE e depois calcular a média de consumo (CNS_ACT_FP)
df_medias = geodf_geral[['NRO_CLIENT', 'CNS_ACT_FP']].groupby('NRO_CLIENT').mean().reset_index()
df_medias.tail()

,NRO_CLIENT,CNS_ACT_FP
1075,1973184,313.930000
1076,1973304,413.300000
1077,1975643,280.533333
1078,1978734,334.000000
1079,1992716,240.600000


In [ ]:
""""Problema: Temos muitos dados faltantes nesse intervalo de tempo"""